In [0]:
import os
import pprint
import tensorflow as tf

if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)


ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!


In [0]:
!mkdir .kaggle

In [0]:
import json

In [0]:
token={"username":"akshaylucof","key":"2ba90f5960ccb38c72cb2615892af210"}

In [0]:
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [0]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
!kaggle datasets download -d akshaylucof/coviddataset

 75% 9.00M/11.9M [00:00<00:00, 16.1MB/s]
100% 11.9M/11.9M [00:00<00:00, 18.8MB/s]


In [0]:
!unzip /content/{/content}/datasets/akshaylucof/coviddataset/coviddataset.zip -d /content/{

Archive:  /content/{/content}/datasets/akshaylucof/coviddataset/coviddataset.zip
replace /content/{/dataset/Test/covid/nejmoa2001191_f3-PA.jpeg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /content/{/dataset/Test/covid/nejmoa2001191_f3-PA.jpeg  
  inflating: /content/{/dataset/Test/covid/nejmoa2001191_f4.jpeg  
  inflating: /content/{/dataset/Test/covid/nejmoa2001191_f5-PA.jpeg  
  inflating: /content/{/dataset/Test/covid/radiol.2020200490.fig3.jpeg  
  inflating: /content/{/dataset/Test/covid/radiopedia-covid-19-pneumonia-2.jpg  
  inflating: /content/{/dataset/Test/normal/IM-0033-0001-0001.jpeg  
  inflating: /content/{/dataset/Test/normal/IM-0240-0001.jpeg  
  inflating: /content/{/dataset/Test/normal/IM-0466-0001.jpeg  
  inflating: /content/{/dataset/Test/normal/NORMAL2-IM-0315-0001.jpeg  
  inflating: /content/{/dataset/Test/normal/NORMAL2-IM-0696-0001.jpeg  
  inflating: /content/{/dataset/Train/covid/1-s2.0-S0140673620303706-fx1_lrg.jpg  
  inflating: /content/{/datase

In [0]:
from tensorflow.keras.applications.vgg16 import VGG16
img_rows, img_cols = 224, 224 
model = VGG16(weights='imagenet',include_top = False,input_shape = (img_rows, img_cols, 3))

58892288/58889256 [==============================] - 0s 0us/step


In [0]:
for layer in model.layers:
    layer.trainable = False

In [0]:
def addTopModelResNet(bottom_model, num_classes):

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(64,activation='relu')(top_model)
    top_model=Dropout(.2)(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

In [0]:

from tensorflow.keras.preprocessing import image 
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D

num_classes = 2

FC_Head = addTopModelResNet(model, num_classes)

model = Model(inputs = model.input, outputs = FC_Head)

print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [0]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = '/content/{/dataset/Train'
validation_data_dir = '/content/{/dataset/Test'

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
batch_size = 2
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')

Found 40 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


Using TensorFlow backend.


Found 40 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


In [0]:
from tensorflow import keras
from keras.callbacks import ModelCheckpoint, EarlyStopping

                     

model.compile(loss = 'categorical_crossentropy',
              optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001),
              metrics = ['accuracy'])

nb_train_samples = 40
nb_validation_samples = 10

epochs = 25
batch_size = 2

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

Epoch 1/25
20/20 [==============================] - 1s 75ms/step - loss: 0.4384 - accuracy: 0.8500 - val_loss: 0.5076 - val_accuracy: 0.6000
Epoch 2/25
20/20 [==============================] - 1s 74ms/step - loss: 0.3822 - accuracy: 0.8250 - val_loss: 0.2766 - val_accuracy: 1.0000
Epoch 3/25
20/20 [==============================] - 2s 76ms/step - loss: 0.3299 - accuracy: 0.8500 - val_loss: 0.3240 - val_accuracy: 0.9000
Epoch 4/25
20/20 [==============================] - 1s 68ms/step - loss: 0.3589 - accuracy: 0.8500 - val_loss: 0.4177 - val_accuracy: 0.7000
Epoch 5/25
20/20 [==============================] - 1s 72ms/step - loss: 0.3569 - accuracy: 0.8750 - val_loss: 0.3218 - val_accuracy: 0.9000
Epoch 6/25
20/20 [==============================] - 2s 80ms/step - loss: 0.3027 - accuracy: 0.8500 - val_loss: 0.2185 - val_accuracy: 1.0000
Epoch 7/25
20/20 [==============================] - 2s 77ms/step - loss: 0.3745 - accuracy: 0.8000 - val_loss: 0.3330 - val_accuracy: 0.8000
Epoch 8/25
20